In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import os
import sys
import argparse
import torch
from getdist import plots, MCSamples
import getdist
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal

In [2]:
path = os.path.realpath(os.path.join(os.getcwd(), '../..'))
sys.path.insert(0, path)

In [3]:
from nnest.trainer import Trainer

In [4]:
dims = 2
def loglike(z):
    z1 = z[:, 0]
    z2 = z[:, 1]
    return - (z1 ** 2 + z2 - 11.) ** 2 - (z1 + z2 ** 2 - 7.) ** 2
def transform(x):
    return 5. * x

In [5]:
n_samples = 1000
fraction = 0.02

In [6]:
x = 2 * (np.random.uniform(size=(int(n_samples / fraction), 2)) - 0.5)
likes = loglike(transform(x))
idx = np.argsort(-likes)
samples = x[idx[0:n_samples]]

In [7]:
t = Trainer(dims, 128,  num_blocks=5, num_layers=1, scale='')

[nnest.trainer] [INFO] SingleSpeed(
  (net): FlowSequential(
    (0): CouplingLayer(
      (scale_net): Sequential(
        (0): Linear(in_features=2, out_features=128, bias=True)
        (1): Tanh()
        (2): Linear(in_features=128, out_features=128, bias=True)
        (3): Tanh()
        (4): Linear(in_features=128, out_features=2, bias=True)
      )
      (translate_net): Sequential(
        (0): Linear(in_features=2, out_features=128, bias=True)
        (1): ReLU()
        (2): Linear(in_features=128, out_features=128, bias=True)
        (3): ReLU()
        (4): Linear(in_features=128, out_features=2, bias=True)
      )
    )
    (1): CouplingLayer(
      (scale_net): Sequential(
        (0): Linear(in_features=2, out_features=128, bias=True)
        (1): Tanh()
        (2): Linear(in_features=128, out_features=128, bias=True)
        (3): Tanh()
        (4): Linear(in_features=128, out_features=2, bias=True)
      )
      (translate_net): Sequential(
        (0): Linear(in_feat

In [8]:
t.train(samples, jitter=0.1)

[nnest.trainer] [INFO] Number of training samples [1000]
[nnest.trainer] [INFO] Training jitter [0.1000]
[nnest.trainer] [INFO] Epoch [1] train loss [0.0194] validation loss [0.0164]
[nnest.trainer] [INFO] Epoch [50] train loss [0.0031] validation loss [-0.0015]
[nnest.trainer] [INFO] Epoch [100] train loss [0.0024] validation loss [-0.0024]
[nnest.trainer] [INFO] Epoch [150] train loss [0.0017] validation loss [-0.0036]
[nnest.trainer] [INFO] Epoch [200] train loss [0.0015] validation loss [-0.0041]
[nnest.trainer] [INFO] Epoch [250] train loss [0.0015] validation loss [-0.0047]
[nnest.trainer] [INFO] Epoch [300] train loss [0.0011] validation loss [-0.0050]
[nnest.trainer] [INFO] Epoch [350] train loss [0.0008] validation loss [-0.0053]
[nnest.trainer] [INFO] Epoch [400] train loss [0.0000] validation loss [-0.0054]
[nnest.trainer] [INFO] Epoch [450] train loss [0.0001] validation loss [-0.0057]
[nnest.trainer] [INFO] Epoch [500] train loss [0.0000] validation loss [-0.0059]
[nnest.t

In [9]:
x_synth = t.netG.sample(samples.size).detach().cpu().numpy()
z = t.get_latent_samples(samples)

In [10]:
grid = []
for x in np.linspace(np.min(samples[:, 0])*0.8, np.max(samples[:, 0])*1.2, 10):
    for y in np.linspace(np.min(samples[:, 1])*0.8, np.max(samples[:, 1])*1.2, 5000):
        grid.append([x, y])
for y in np.linspace(np.min(samples[:, 1])*0.8, np.max(samples[:, 1])*1.2, 10):
    for x in np.linspace(np.min(samples[:, 0])*0.8, np.max(samples[:, 0])*1.2, 5000):
        grid.append([x, y])
grid = np.array(grid)
z_grid = t.get_latent_samples(grid)

In [11]:
fig, ax = plt.subplots(1, 3, figsize=(12, 5))
ax[0].scatter(grid[:, 0], grid[:, 1], c=grid[:, 0], marker='.', s=1, linewidths=0)
ax[0].scatter(samples[:, 0], samples[:, 1], s=4)
ax[0].set_title('Real data')
ax[1].scatter(z_grid[:, 0], z_grid[:, 1], c=grid[:, 0], marker='.', s=1, linewidths=0)
ax[1].scatter(z[:, 0], z[:, 1], s=4)
ax[1].set_title('Latent data')
ax[1].set_xlim([-np.max(np.abs(z)), np.max(np.abs(z))])
ax[1].set_ylim([-np.max(np.abs(z)), np.max(np.abs(z))])
ax[2].scatter(x_synth[:, 0], x_synth[:, 1], s=2)
ax[2].set_title('Synthetic data')
plt.tight_layout()
plt.show()

/Users/ppzam/anaconda/envs/flows/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  del sys.path[0]
